In [1]:
import http.client
import config
import pandas as pd
import json
import folium

In [ ]:
def get_arrivals(airlines, dates):
    conn = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")
    conn2 = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': f"{config.api_key}",
        'X-RapidAPI-Host': "flight-fare-search.p.rapidapi.com"
    }

    df_arrivals = pd.DataFrame()  # Initialize an empty DataFrame
    df_departures = pd.DataFrame()  # Initialize an empty DataFrame

    for airline in airlines:
        for date in dates:
            conn.request("GET", f"/v2/airport/arrivals?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)
            conn2.request("GET", f"/v2/airport/departures?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)

            arrivals = conn.getresponse().read().decode('utf-8')
            departures = conn2.getresponse().read().decode('utf-8')

            arrivals = json.loads(arrivals)
            departures = json.loads(departures)

            arrivals_data = arrivals['result']['flights']
            departures_data = departures['result']['flights']

            df_arrivals_temp = pd.json_normalize(arrivals_data)  # Temporary DataFrame for each request
            df_departures_temp = pd.json_normalize(departures_data)  # Temporary DataFrame for each request

            df_arrivals = pd.concat([df_arrivals, df_arrivals_temp])  # Concatenate the temporary DataFrame with the main one

    return df_arrivals, df_departures

In [2]:
conn = http.client.HTTPSConnection("app.goflightlabs.com")
conn2 = http.client.HTTPSConnection("app.goflightlabs.com")
conn3 = http.client.HTTPSConnection("app.goflightlabs.com")
conn4 = http.client.HTTPSConnection("app.goflightlabs.com")

conn.request("GET", f"/flights?access_key={config.api_key}&depIata=ATL")
conn2.request("GET", f"/flights?access_key={config.api_key}&arrIata=ATL")
conn3.request("GET", f"/historical/2023-05-10?access_key={config.api_key}&code=ATL&type=departure")
conn4.request("GET", f"/historical/2023-05-10?access_key={config.api_key}&code=ATL&type=arrival")

real_time_departures = conn.getresponse().read().decode('utf-8')
real_time_departures = json.loads(real_time_departures)

real_time_arrivals = conn2.getresponse().read().decode('utf-8')
real_time_arrivals = json.loads(real_time_arrivals)

historical_departures = conn3.getresponse().read().decode('utf-8')
historical_departures = json.loads(historical_departures)

historical_arrivals = conn4.getresponse().read().decode('utf-8')
historical_arrivals = json.loads(historical_arrivals)

In [3]:
df_real_time_departures = pd.json_normalize(real_time_departures['data'])
df_real_time_arrials = pd.json_normalize(real_time_arrivals['data'])
df_real_time_flights = pd.concat([df_real_time_departures, df_real_time_arrials])
df_hist_departures = pd.json_normalize(historical_departures['data'])
df_hist_arrivals = pd.json_normalize(historical_arrivals['data'])
df_hist_flights = pd.concat([df_hist_departures, df_hist_arrivals])

In [5]:
df_hist_flights.head()

,type,status,departure.iataCode,departure.icaoCode,departure.terminal,departure.gate,departure.delay,departure.scheduledTime,departure.estimatedTime,departure.actualTime,...,codeshared.airline.name,codeshared.airline.iataCode,codeshared.airline.icaoCode,codeshared.flight.number,codeshared.flight.iataNumber,codeshared.flight.icaoNumber,arrival.delay,arrival.actualTime,arrival.estimatedRunway,arrival.actualRunway
0,departure,active,atl,katl,i,f6,23.0,2023-05-09t20:00:00.000,2023-05-09t20:10:00.000,2023-05-09t20:22:00.000,...,delta air lines,dl,dal,176,dl176,dal176,NaN,NaN,NaN,NaN
1,departure,active,atl,katl,i,f6,23.0,2023-05-09t20:00:00.000,2023-05-09t20:10:00.000,2023-05-09t20:22:00.000,...,delta air lines,dl,dal,176,dl176,dal176,NaN,NaN,NaN,NaN
2,departure,active,atl,katl,i,f6,23.0,2023-05-09t20:00:00.000,2023-05-09t20:10:00.000,2023-05-09t20:22:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,departure,active,atl,katl,s,b28,15.0,2023-05-09t20:00:00.000,2023-05-09t20:00:00.000,2023-05-09t20:14:00.000,...,delta air lines,dl,dal,1280,dl1280,dal1280,NaN,NaN,NaN,NaN
4,departure,active,atl,katl,s,b28,15.0,2023-05-09t20:00:00.000,2023-05-09t20:00:00.000,2023-05-09t20:14:00.000,...,delta air lines,dl,dal,1280,dl1280,dal1280,NaN,NaN,NaN,NaN


In [8]:
l = ['departure.scheduledTime', 'departure.estimatedTime', 'departure.actualTime', 'departure.estimatedRunway', 'departure.actualRunway', 'arrival.scheduledTime', 'arrival.estimatedTime', 'arrival.actualTime', 'arrival.estimatedRunway', 'arrival.actualRunway']
for i in l:
    df_hist_flights[i] = pd.to_datetime(df_hist_flights[i])


df_hist_flights

,type,status,departure.iataCode,departure.icaoCode,departure.terminal,departure.gate,departure.delay,departure.scheduledTime,departure.estimatedTime,departure.actualTime,...,codeshared.airline.name,codeshared.airline.iataCode,codeshared.airline.icaoCode,codeshared.flight.number,codeshared.flight.iataNumber,codeshared.flight.icaoNumber,arrival.delay,arrival.actualTime,arrival.estimatedRunway,arrival.actualRunway
0,departure,active,atl,katl,i,f6,23.0,2023-05-09 20:00:00,2023-05-09 20:10:00,2023-05-09 20:22:00,...,delta air lines,dl,dal,176,dl176,dal176,NaN,NaT,NaT,NaT
1,departure,active,atl,katl,i,f6,23.0,2023-05-09 20:00:00,2023-05-09 20:10:00,2023-05-09 20:22:00,...,delta air lines,dl,dal,176,dl176,dal176,NaN,NaT,NaT,NaT
2,departure,active,atl,katl,i,f6,23.0,2023-05-09 20:00:00,2023-05-09 20:10:00,2023-05-09 20:22:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3,departure,active,atl,katl,s,b28,15.0,2023-05-09 20:00:00,2023-05-09 20:00:00,2023-05-09 20:14:00,...,delta air lines,dl,dal,1280,dl1280,dal1280,NaN,NaT,NaT,NaT
4,departure,active,atl,katl,s,b28,15.0,2023-05-09 20:00:00,2023-05-09 20:00:00,2023-05-09 20:14:00,...,delta air lines,dl,dal,1280,dl1280,dal1280,NaN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3426,arrival,landed,srq,ksrq,NaN,b6,11.0,2023-05-10 16:57:00,NaT,2023-05-10 17:08:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-10 18:13:00,2023-05-10 18:13:00,2023-05-10 18:13:00
3427,arrival,unknown,bwi,kbwi,NaN,d29,192.0,2023-05-10 17:00:00,2023-05-10 20:12:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3428,arrival,landed,dfw,kdfw,NaN,NaN,NaN,2023-05-10 16:25:00,NaT,2023-05-10 16:20:00,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2023-05-10 19:00:00,2023-05-10 19:00:00,2023-05-10 19:00:00
3429,arrival,landed,cvg,kcvg,NaN,NaN,29.0,2023-05-10 17:30:00,NaT,2023-05-10 17:59:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-10 19:06:00,2023-05-10 19:06:00,2023-05-10 19:06:00


In [115]:
# df_departures.rename(columns={'time':'Time_UTC',
#                               'operator':'Operator',
#                               'departure.time.timeAMPM':'Departure_Time_AM/PM',
#                               'departure.time.time24':'Departure_Time_24',
#                               'departure.airport.iata':'Arrival_Airport_IATA',
#                               'departure.airport.city':'Arrival_Airport_City',
#                               'arrival.timeAMPM':'Arrival_Time_AM/PM',
#                               'arrival.time24':'Arrival_Time_24',
#                               'carrier.code':'Airline_Code',
#                               'carrier.name':'Airline_Name',
#                               'carrier.flightNumber':'Airline_Flight_Number',
#                               'date':'Date_UTC',
#                               'time_EST':'Time_EST',
#                               'date_EST':'Date_EST'}, inplace=True)
#
# df_departures = df_departures[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Arrival_Airport_IATA', 'Arrival_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]

In [116]:
# df_arrivals.rename(columns={'time':'Time_UTC',
#                               'operator':'Operator',
#                               'departure.time.timeAMPM':'Departure_Time_AM/PM',
#                               'departure.time.time24':'Departure_Time_24',
#                               'departure.airport.iata':'Departure_Airport_IATA',
#                               'departure.airport.city':'Departure_Airport_City',
#                               'arrival.timeAMPM':'Arrival_Time_AM/PM',
#                               'arrival.time24':'Arrival_Time_24',
#                               'carrier.code':'Airline_Code',
#                               'carrier.name':'Airline_Name',
#                               'carrier.flightNumber':'Airline_Flight_Number',
#                               'date':'Date_UTC',
#                               'time_EST':'Time_EST',
#                               'date_EST':'Date_EST'}, inplace=True)
#
# df_arrivals = df_arrivals[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Departure_Airport_IATA', 'Departure_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]